In [1]:
import numpy as np
import torch
import torch.nn as nn
import time
from tqdm import *
import os
import argparse
from GenerateData import *

In [2]:
import torch
CUDA_ORDER=0
DEVICE = torch.device(f"cuda:{CUDA_ORDER}" if torch.cuda.is_available() else "cpu")
print(f"当前启用 {DEVICE}")

当前启用 cuda:0


In [3]:
# Parser
parser = argparse.ArgumentParser(description='DFVM')
parser.add_argument('--dimension', type=int, default=100, metavar='N',
                    help='dimension of the problem (default: 100)')
parser.add_argument('--seed', type=int, default=0, metavar='N',
                    help='random seed (default: 0)')
seed = parser.parse_args().seed
# Omega space domain
DIMENSION = parser.parse_args().dimension
a = [0 for _ in range(DIMENSION)]
b = [1 for _ in range(DIMENSION)]

# Finite Volume
EPSILON = 1e-3         # Domain size
BDSIZE = 1

# Network
DIM_INPUT = DIMENSION   # Input dimension
NUM_UNIT = 40           # Number of neurons in a single layer
DIM_OUTPUT = 1          # Output dimension
NUM_LAYERS = 6          # Number of layers in the model

# Optimizer
IS_DECAY = 0
LEARN_RATE = 1.1e-3      # Learning rate
LEARN_FREQUENCY = 50     # Learning rate change interval
LEARN_LOWWER_BOUND = 1e-5
LEARN_DECAY_RATE = 0.99
LOSS_FN = nn.MSELoss()

# Training
CUDA_ORDER = "0"
NUM_TRAIN_SMAPLE = 10000  # Size of the training set
NUM_TRAIN_TIMES = 1       # Number of training samples
NUM_ITERATION = 20000     # Number of training iterations per sample

# Re-sampling
IS_RESAMPLE = 0
SAMPLE_FREQUENCY = 2000   # Re-sampling interval

# Testing
NUM_TEST_SAMPLE = 10000
TEST_FREQUENCY = 1        # Output interval

# Loss weight
BETA = 1000               # Weight of the boundary loss function

# Save model
IS_SAVE_MODEL = 1



/home/young/anaconda3/envs/pytorch/bin/python


In [ ]:
def train_pipeline():
    # define device
    DEVICE = torch.device(f"cuda:{CUDA_ORDER}" if torch.cuda.is_available() else "cpu")
    #DEVICE = torch.device("cpu")
    print(f"当前启用 {DEVICE}")
    # define equation
    Eq = PoissonEQuation(DIMENSION, EPSILON, BDSIZE, DEVICE)
    # define model
    # torch.set_default_dtype(torch.float64)
    model = MLP(DIM_INPUT, DIM_OUTPUT, NUM_UNIT).to(DEVICE)
    optA   = torch.optim.Adam(model.parameters(), lr=LEARN_RATE) 
    solver = DFVMsolver(Eq, model, DEVICE)

    x      = Eq.interior(NUM_TRAIN_SMAPLE)
    x_bd   = Eq.neighborhoodBD(x)
    print(x_bd.shape)
    int_f  = solver.integrate_F(x)
    bd_dir = Eq.outerNormalVec()
    x_boundary = Eq.boundary(100)
    
    # 网络迭代
    elapsed_time     = 0    # 计时
    training_history = []    # 记录数据

    for step in tqdm(range(NUM_ITERATION+1)):
        if IS_DECAY and step and step % LEARN_FREQUENCY == 0:
            for p in optA.param_groups:
                if p['lr'] > LEARN_LOWWER_BOUND:
                    p['lr'] = p['lr']*LEARN_DECAY_RATE
                    print(f"Learning Rate: {p['lr']}")

        start_time = time.time()
        int_bd   = solver.integrate_BD(x, x_bd, bd_dir).reshape(-1,1)
        loss_int = LOSS_FN(-int_bd, int_f)
        loss_bd  = solver.loss_boundary(x_boundary)
        loss     = loss_int + BETA*loss_bd 
        optA.zero_grad()
        loss.backward()
        optA.step()
        
        epoch_time = time.time() - start_time
        elapsed_time = elapsed_time + epoch_time
        if step % TEST_FREQUENCY == 0:
            loss_int     = loss_int.cpu().detach().numpy()
            loss_bd      = loss_bd.cpu().detach().numpy()
            loss         = loss.cpu().detach().numpy()
            L2error,ME,T = solver.TEST(NUM_TEST_SAMPLE)
            if step and step%1000 == 0:
                tqdm.write( f'\nStep: {step:>5}, '
                            f'Loss_int: {loss_int:>10.5f}, '
                            f'Loss_bd: {loss_bd:>10.5f}, '
                            f'Loss: {loss:>10.5f}, '                                     
                            f'L2 error: {L2error:.5f}, '                                     
                            f'Time: {elapsed_time:.2f}')
            training_history.append([step, L2error, ME, loss, elapsed_time, epoch_time, T])

    training_history = np.array(training_history)
    print(np.min(training_history[:,1]))
    print(np.min(training_history[:,2]))

    save_time = time.localtime()
    save_time = f'[{save_time.tm_mday:0>2d}{save_time.tm_hour:0>2d}{save_time.tm_min:0>2d}]'
    dir_path  = os.getcwd() + f'/PossionEQ_seed{seed}/'
    file_name = f'{DIMENSION}DIM-DFVM-{BETA}weight-{NUM_ITERATION}itr-{EPSILON}R-{BDSIZE}bd-{LEARN_RATE}lr.csv'
    file_path = dir_path + file_name

    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    np.savetxt(file_path, training_history,
                delimiter =",",
                header    ="step, L2error, MaxError, loss, elapsed_time, epoch_time, inference_time",
                comments  ='')
    print('Training History Saved!')

    if IS_SAVE_MODEL:
        torch.save(model.state_dict(), dir_path + f'{DIMENSION}DIM-DFVM_net')
        print('DFVM Network Saved!')


if __name__ == "__main__":
    setup_seed(seed)
    train_pipeline()